<a href="https://colab.research.google.com/github/dakyommii/MinhwaProject/blob/main/minhwa-explanation/train/kogpt2-train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoGPT2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Library

In [2]:
!pip install transformers torch

In [3]:
!pip install pandas

## dataset

In [ ]:
import pandas as pd

### 데이터셋 불러오기

In [ ]:
import csv

# CSV 파일 경로 및 파일명 설정
file_path = "/content/drive/MyDrive/seol/datasets/kor_data-v4.csv"  # 파일 경로와 파일명을 적절히 수정하세요.

# 각 열을 저장할 빈 리스트 생성
keyword_lst = []
text_lst = []  # 필요한 만큼 열을 추가

# CSV 파일 읽기
with open(file_path, "r", newline="") as csv_file:
    csv_reader = csv.reader(csv_file)

    # 헤더를 건너뛰기
    next(csv_reader)

    # 각 행을 읽어와서 열별로 데이터를 리스트에 추가
    for row in csv_reader:
        if len(row) >= 2:  # 예를 들어, 3개의 열이 있는 경우
            keyword_lst.append(row[0])
            text_lst.append(row[2])

In [ ]:
# 특정 단어 포함한 텍스트를 조합하여 데이터 증강
from itertools import combinations

keywords = ['white flower', 'red flower', 'yellow flower', 'lotus', 'rock', 'bird', 'brown moran', 'white moran']
genre='Morando'

# 가능한 모든 조합 생성
all_combinations = []
for r in range(1, len(keywords) + 1):  # 1개부터 모든 키워드를 사용한 조합까지 생성
    combinations_r = combinations(keywords, r)
    all_combinations.extend(combinations_r)

# 생성된 조합 출력
combos=[]
for combo in all_combinations:
  s=""
  for i in range(0,len(combo)):
    if(s!=""): s+=','
    s+=combo[i]

  combos.append(s)

unique_combo = list(set(combos))

genre_list = [genre] * len(unique_combo)


In [ ]:
#데이터 전처리
for i in range(0,len(text_lst)):
  val=text_lst[i]
  val_new = val.replace('핑크', '분홍색') # 바꿀 텍스트와 대체할 텍스트
  text_lst[i]=val_new

In [ ]:
# 데이터를 담을 딕셔너리 생성
data = {
    'keyword': unique_combo,
    'explanation': exp
}

# 딕셔너리를 데이터프레임으로 변환
df = pd.DataFrame(data)

# 데이터프레임을 CSV 파일로 저장
csv_file_path = "sample_data.csv"  # 저장할 CSV 파일의 경로 및 파일명 지정
df.to_csv(csv_file_path, index=False)  # index=False로 설정하면 인덱스를 CSV에 저장하지 않습니다.


In [ ]:
import pandas as pd

# CSV 파일 로드
csv_file_path = "sample_data.csv"  # CSV 파일 경로를 지정해야 합니다.
df = pd.read_csv(csv_file_path)

# 데이터 확인
print(df.head())  # 데이터프레임의 처음 몇 행을 출력합니다.

    Name  Age         City
0   John   25     New York
1  Alice   30  Los Angeles
2    Bob   35      Chicago


## fine-tuning

In [ ]:
from typing import Optional
import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast
from fastai.text.all import *
import fastai
import re

### 모델 로드

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/seol/ckpt/kogpt2-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') # 토크나이저
model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/seol/ckpt/kogpt2-v2")  # 모델 불러오기

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1499: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


### 데이터셋 로드

In [ ]:
import pandas as pd
import csv

# CSV 파일 로드
csv_file_path = "/content/drive/MyDrive/seol/datasets/kor_data-v4.csv"
df = pd.read_csv(csv_file_path)

# 텍스트 데이터 추출
keyword_col = df['keyword']
exp_col = df['explanation']
# 다른 열도 필요한 만큼 추가

# 추출된 텍스트 데이터 리스트를 리스트로 변환
kwd_lst = keyword_col.tolist()
exp_lst=exp_col.tolist()

# 파인튜닝에 사용할 데이터 생성
train_texts = []
for i in range(len(kwd_lst)):
    combined_text = f"{kwd_lst[i]} | {exp_lst[i]}"  # 필요에 따라 조합
    train_texts.append(combined_text)

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader

In [4]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader

# 데이터셋 클래스 정의
class MyDataset(Dataset):
    def __init__(self, tokenizer, text_list, max_length=512):
        self.tokenizer = tokenizer
        self.text_list = text_list
        self.max_length = max_length

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, idx):
        text = self.text_list[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        inputs = {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask']  # attention_mask 추가
        }
        return inputs


# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델과 토크나이저 로드
model.to(device)

# 파인튜닝을 위한 데이터 준비
train_dataset = MyDataset(tokenizer, train_texts)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# 옵티마이저 및 스케줄러 설정
num_epochs = 200
optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=100, num_training_steps=len(train_dataloader) * num_epochs
)

# 초기 손실을 큰 값으로 설정
best_loss = float("inf")

# 파인튜닝 하기
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch in train_dataloader:
      inputs = {
          'input_ids': batch['input_ids'].to(device),
          'attention_mask': batch['attention_mask'].to(device)
      }
      outputs = model(**inputs, labels=inputs['input_ids'])  # labels도 input_ids와 동일하게 설정
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()
      total_loss += loss.item()

    # 평균 손실 계산
    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}")

    # 현재 모델의 손실이 이전 모델보다 낮으면 모델 저장
    if average_loss < best_loss:
        best_loss = average_loss
        model.save_pretrained("/content/drive/MyDrive/seol/ckpt/kogpt2-v2")
        tokenizer.save_pretrained("/content/drive/MyDrive/seol/ckpt/kogpt2-v2")


### 텍스트 생성 테스트

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_path = "/content/drive/MyDrive/ckpt/kogpt2"  # 모델과 토크나이저를 동일한 디렉토리에 저장한 경우
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') # 토크나이저

prompt = "tiger,red flower" ## 입력 텍스트
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# 텍스트 생성
output = model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

# 생성된 텍스트 출력
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)
